## Import libraries

In [1]:
import os
import sys

import pandas as pd
import geopandas as gpd
import osmnx as ox
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

module_path = os.path.abspath(os.path.join('../../'))
if module_path not in sys.path:
    sys.path.append(module_path)
    import aup

/home/jovyan/accesibilidad-urbana/aup/data.py:24: UserWarning: The `utils.config` function is deprecated and will be removed in a future release. Instead, use the `settings` module directly to configure a global setting's value. For example, `ox.settings.log_console=True`.
  ox.config(


## 00 - Required data

In [7]:
# Location of area of interest (Required)
aoi_dir = "../../data/external/temporal_todocker/arroyo_vivo/poligono_proyectoarroyovivo_32614.gpkg"

# Name of points of interest (Required)
pois_name = 'mercados'
# Location of points of interest (Required)
pois_dir = "../../data/external/temporal_todocker/arroyo_vivo/mercados_pulgas_tianguis_cuenca_32614.gpkg"

## 01 - Find nodes proximity to pois in aoi

In [9]:
def find_nodes_proximity(aoi_dir,pois_dir,pois_name):

    #-------------------------------------------------- Create OSMnx network

    # Read area of interest (aoi)
    aoi = gpd.read_file(aoi_dir)

    # Create OSMnx network for area of interest
    G, nodes, edges = aup.create_osmnx_network(aoi,how='from_bbox',network_type='all_private')

    #-------------------------------------------------- Assign nearest node(osmid) to each point of interest

    # Load pois
    pois = gpd.read_file(pois_dir)
    pois = pois.set_crs("EPSG:4326")

    # Filter pois
    pois_aoi = gpd.sjoin(pois,aoi,how='inner')

    # Format to calculate nearest
    nodes_gdf = nodes.set_crs("EPSG:4326")
    edges_gdf = edges.set_crs("EPSG:4326")
    nodes_gdf = nodes_gdf.set_index('osmid')
    edges_gdf = edges_gdf.set_index(["u", "v", "key"])

    # Calculate nearest
    nearest = aup.find_nearest(G, nodes_gdf, pois, return_distance= True)
    print("Calculated distances from pois to nearest node.")

    #-------------------------------------------------- Calculate distance from each node to nearest poi
    # Format
    edges_gdf['length'].fillna(edges_gdf['length'].mean(),inplace=True)

    # ELEMENTS NEEDED OUTSIDE THE LOOP
    # df_temp: Each column will store a batch of procesed nodes.
    df_temp = nodes_gdf.copy()

    # ELEMENTS NEEDED OUTSIDE THE LOOP
    # nodes_analysis is a nodes_gdf (index reseted) used in aup.calculate_distance_nearest_poi
    nodes_analysis = nodes_gdf.reset_index().copy()

    # ELEMENTS NEEDED OUTSIDE THE LOOP
    #nodes_distance: Minimum time/distance found in all batches will be added from df_min (within if/elif/else) to nodes_distance 
    # (which keeps x,y and geometry data)
    nodes_distance = nodes_gdf.copy()

    s = pois_name

    if len(nearest) % 250:
        batch_size = len(nearest)/200
        for k in range(int(batch_size)+1):
            #---------------------------------------------------------------------------------------------------------------------------------------------
            print(f"Starting range k = {k+1} of {int(batch_size)+1} for source {s}.")
            #---------------------------------------------------------------------------------------------------------------------------------------------
            source_process = nearest.iloc[int(200*k):int(200*(1+k))].copy()
            nodes_distance_prep = aup.calculate_distance_nearest_poi(source_process, nodes_analysis, edges_gdf, s, 'osmid', wght='length')

            #A middle gdf is created whose columns will be the name of the amenity and the batch number it belongs to
            df_int = pd.DataFrame()
            df_int['dist_'+str(k)+s] = nodes_distance_prep['dist_'+s]

            #The middle gdf is merged into the previously created temporary gdf to store the data by node, each batch in a column.
            df_temp = df_temp.merge(df_int, left_index=True, right_index=True)

        # Once finished, drop the non-distance values from the temporary gdf
        df_temp.drop(['x', 'y', 'street_count','geometry'], inplace = True, axis=1)

        #We apply the min function to find the minimum value. This value is sent to a new df_min
        df_min = pd.DataFrame()
        df_min['dist_'+s] = df_temp.min(axis=1)

        #We merge df_min which contains the shortest distance to the POI with nodes_distance which will store all final data
        nodes_distance = nodes_distance.merge(df_min, left_index=True, right_index=True)

        #Final data gets converted to time, assuming a walking speed of 4km/hr
        nodes_time = nodes_distance.copy()
        nodes_time['time'] = (nodes_time['dist_'+s]*60)/4000

    else:
        batch_size = len(nearest)/250
        for k in range(int(batch_size)+1):
            #---------------------------------------------------------------------------------------------------------------------------------------------
            print(f"Starting range k = {k+1} of {int(batch_size)+1} for source {s}.")
            #---------------------------------------------------------------------------------------------------------------------------------------------
            source_process = nearest.iloc[int(250*k):int(250*(1+k))].copy()
            nodes_distance_prep = aup.calculate_distance_nearest_poi(source_process, nodes_analysis, edges_gdf, s, 'osmid', wght='length')

            #A middle gdf is created whose columns will be the name of the amenity and the batch number it belongs to
            df_int = pd.DataFrame()
            df_int['dist_'+str(k)+s] = nodes_distance_prep['dist_'+s]

            #The middle gdf is merged into the previously created temporary gdf to store the data by node, each batch in a column.
            df_temp = df_temp.merge(df_int, left_index=True, right_index=True)

        # Once finished, drop the non-distance values from the temporary gdf
        df_temp.drop(['x', 'y', 'street_count','geometry'], inplace = True, axis=1)

        #We apply the min function to find the minimum value. This value is sent to a new df_min
        df_min = pd.DataFrame()
        df_min['dist_'+s] = df_temp.min(axis=1)

        #We merge df_min which contains the shortest distance to the POI with nodes_distance which will store all final data
        nodes_distance = nodes_distance.merge(df_min, left_index=True, right_index=True)

        #Final data gets converted to time, assuming a walking speed of 4km/hr
        nodes_time = nodes_distance.copy()
        nodes_time['time'] = (nodes_time['dist_'+s]*60)/4000

    #Format nodes_time
    nodes_time['source'] = s
    nodes_time.reset_index(inplace=True)
    nodes_time = nodes_time.set_crs("EPSG:4326")
    nodes_time = nodes_time[['osmid','time','source','x','y','geometry']]

    return nodes_time

In [8]:
nodes_time = find_nodes_proximity(aoi_dir,pois_dir,pois_name)
nodes_time

AttributeError: module 'aup' has no attribute 'create_osmnx_network'

## 01 - Transform nodes data

In [26]:
s = nodes_time.source.unique()[0]

nodes_analysis = nodes_time[['osmid','geometry','time']]
nodes_analysis.rename(columns={'time':s},inplace=True)

nodes_analysis.head(2)

/tmp/ipykernel_24760/3882504483.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  nodes_analysis.rename(columns={'time':s},inplace=True)


,osmid,geometry,random_pois
0,301189389,POINT (-102.34221 21.84854),6.400245
1,301189406,POINT (-102.35022 21.85082),17.556360


## 02 - Group by hex

In [7]:
aoi = gpd.read_file(aoi_dir)
pop_dir = "../../data/external/prox_latam/pop_gdf_ags.gpkg"
index_column = 'cvegeo'
pop_columns = ['pobtot','p_0a2','p_3a5','p_60ymas']
res_list = [8,9]

hex_socio_gdf = aup.create_popdata_hexgrid(aoi,pop_dir,index_column,pop_columns,res_list)

Filtered pop_gdf.
Extracted pop_gdf centroids.
Converted to centroids with 1043905 pop vs 1043905 pop in original gdf.
Created hex_grid with 8 resolution
Agregated socio data to hex with a total of 1043905.0 population for resolution 8.
Calculated an average density of 39.75306755993063
Created hex_grid with 9 resolution
Agregated socio data to hex with a total of 1043905.0 population for resolution 9.
Calculated an average density of 57.50817597082862
Finished calculating population by hexgrid for res [8, 9].


In [11]:
hex_socio_gdf.head(2)

,hex_id,geometry,res,pobtot,p_0a2,p_3a5,p_60ymas,dens_pob_ha
0,88498eacb1fffff,"POLYGON ((-102.29640 21.84650, -102.29181 21.8...",8,12230.0,510.0,598.0,1608.0,141.581209
1,88498e34e1fffff,"POLYGON ((-102.25996 21.97565, -102.25536 21.9...",8,103.0,3.0,14.0,4.0,1.192333


In [9]:
nodes_analysis_filter = nodes_analysis.copy()

In [12]:
pop_output = True
index_column = s

In [13]:
hex_idx = gpd.GeoDataFrame()

for res in res_list:
    
    #/////////////////////////////////////////////////////////////////////////// HEXGRID DEPENDS ON POP DATA BEING CALCULATED OR NOT ///////////////////////////////////////////////////////////////////////////
    # If pop_output is true, loads previously created hexgrid with pop data
    if pop_output:
        # Load hexgrid
        hex_pop = hex_socio_gdf.loc[hex_socio_gdf['res'] == res]
        
        #Function group_by_hex_mean requires ID to include resolution
        hex_pop.rename(columns={'hex_id':f'hex_id_{res}'},inplace=True)
        
        # Create hex_tmp
        hex_pop = hex_pop.set_crs("EPSG:4326")
        hex_tmp = hex_pop[[f'hex_id_{res}','geometry']].copy()
        #-------------------------------------------------------------------------------------------------------------------------------------------------
        print(f"Loaded pop hexgrid of resolution {res}")
        #-------------------------------------------------------------------------------------------------------------------------------------------------
        
    # If pop_output is false, creates hexgrid
    else:
        # Create hexgrid (which already has ID_res)
        hexgrid = aup.create_hexgrid(aoi,res)
        
        # Create hex_tmp
        hexgrid = hexgrid.set_crs("EPSG:4326")
        hex_tmp = hexgrid.copy()
        #-------------------------------------------------------------------------------------------------------------------------------------------------
        print(f"Created hexgrid of resolution {res}")
        #-------------------------------------------------------------------------------------------------------------------------------------------------
        
    #/////////////////////////////////////////////////////////////////////////// HEXGRID DEPENDS ON POP DATA BEING CALCULATED OR NOT ///////////////////////////////////////////////////////////////////////////
    
    # group data by hex
    hex_res_idx = aup.group_by_hex_mean(nodes_analysis_filter, hex_tmp, res, index_column)
    hex_res_idx = hex_res_idx.loc[hex_res_idx[index_column]>0].copy()
    
    #-----------------------------------------------------------------------------------------------------------------------------------------------------
    print(f"Grouped nodes data by hexagons res {res}")
    #-----------------------------------------------------------------------------------------------------------------------------------------------------
    
    #////////////////////////////////////////////////////////////////////////////////// ADD POP DATA IF POP DATA IS CONSIDERED /////////////////////////////////////////////////////////////////////////////////
    # Add pop data
    if pop_output:
        pop_list = [f'hex_id_{res}','pobtot','dens_pob_ha']
        hex_res_pop = pd.merge(hex_res_idx, hex_pop[pop_list], on=f'hex_id_{res}')
    else:
        hex_res_pop = hex_res_idx.copy()
    #////////////////////////////////////////////////////////////////////////////////// ADD POP DATA IF POP DATA IS CONSIDERED /////////////////////////////////////////////////////////////////////////////////    
    
    # After funtion group_by_hex_mean we can remove res from ID and set as a column
    hex_res_pop.rename(columns={f'hex_id_{res}':'hex_id'},inplace=True)
    hex_res_pop['res'] = res

    hex_idx = hex_idx.append(hex_res_pop)
        
    #-----------------------------------------------------------------------------------------------------------------------------------------------------    
    print(f"Saved grouped data by hexagons res {res}")
    #-----------------------------------------------------------------------------------------------------------------------------------------------------
    

/tmp/ipykernel_24760/156202181.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hex_pop.rename(columns={'hex_id':f'hex_id_{res}'},inplace=True)


Loaded pop hexgrid of resolution 8
Grouped nodes data by hexagons res 8
Saved grouped data by hexagons res 8
Loaded pop hexgrid of resolution 9


/tmp/ipykernel_24760/156202181.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hex_pop.rename(columns={'hex_id':f'hex_id_{res}'},inplace=True)


Grouped nodes data by hexagons res 9
Saved grouped data by hexagons res 9


In [18]:
hex_idx.rename(columns={s:'max_time'},inplace=True)

In [21]:
# Show
print(hex_idx.shape)
hex_idx.head(2)

(1765, 6)


,hex_id,geometry,max_time,pobtot,dens_pob_ha,res
0,88498eacb1fffff,"POLYGON ((-102.29640 21.84650, -102.29181 21.8...",5.246933,12230.0,141.581209,8
1,88498e34e1fffff,"POLYGON ((-102.25996 21.97565, -102.25536 21.9...",26.711324,103.0,1.192333,8


## 03 - Save

In [ ]:
'../../data/external/temporal_fromjupyter/proximity_function02.gpkg'
hex_idx.to_file(proximityanalysis_save_dir, driver='GPKG')